In [8]:
#!pip install sentinelhub
#!pip install geopandas
#!sudo pip3 install eo-learn

In [199]:
from helper import *

In [200]:
#input_task = S2L1CWCSInput('1_TRUE_COLOR', resx='50m', resy='50m', maxcc=.5,instance_id="f9ffdffa-be54-4758-a111-be29f5331746")
#input_task = S2L1CWCSInput('1_TRUE_COLOR', resx='50m', resy='50m', maxcc=1, instance_id="f9ffdffa-be54-4758-a111-be29f5331746")
o3 = S5PL2CWCSInput('O3', resx='1000m', resy='1000m', maxcc=1,instance_id="73f6b3e5-903b-4689-a1d1-7fbb6918d811")
so2 = S5PL2CWCSInput('SO2', resx='1000m', resy='1000m', maxcc=0.5,instance_id="73f6b3e5-903b-4689-a1d1-7fbb6918d811")
ch4 = S5PL2CWCSInput('CH4', resx='1000m', resy='1000m', maxcc=0.5,instance_id="73f6b3e5-903b-4689-a1d1-7fbb6918d811")
no2 = S5PL2CWCSInput('NO2', resx='1000m', resy='1000m', maxcc=0.5,instance_id="73f6b3e5-903b-4689-a1d1-7fbb6918d811")
hcho = S5PL2CWCSInput('HCHO', resx='1000m', resy='1000m', maxcc=0.5,instance_id="73f6b3e5-903b-4689-a1d1-7fbb6918d811")
co = S5PL2CWCSInput('CO', resx='1000m', resy='1000m', maxcc=0.5,instance_id="73f6b3e5-903b-4689-a1d1-7fbb6918d811")
aero = S5PL2CWCSInput('AER_AI_354_388', resx='1000m', resy='1000m', maxcc=0.5,instance_id="73f6b3e5-903b-4689-a1d1-7fbb6918d811")

In [4]:
time_interval = ['2019-10-20','2019-11-10']

In [5]:
workflow = LinearWorkflow(o3,so2,ch4,no2,hcho,co,aero)

In [6]:
chennai_outline = gpd.read_file("https://raw.githubusercontent.com/gok03/eo-learn-5p-support/master/chennai.geojson")
delhi_outline = gpd.read_file("https://raw.githubusercontent.com/gok03/eo-learn-5p-support/master/delhi.geojson")
hyderbad_outline = gpd.read_file("https://raw.githubusercontent.com/gok03/eo-learn-5p-support/master/hyderabad.geojson")
kolkata_outline = gpd.read_file("https://raw.githubusercontent.com/gok03/eo-learn-5p-support/master/kolkata.geojson")
mumbai_outline = gpd.read_file("https://raw.githubusercontent.com/gok03/eo-learn-5p-support/master/mumbai.geojson")
bangalore_outline = gpd.read_file("https://raw.githubusercontent.com/gok03/eo-learn-5p-support/master/bangalore.geojson")

In [7]:
outline_bbox = get_bbox(delhi_outline)

In [9]:
# result = workflow.execute({o3: {'bbox': outline_bbox, 'time_interval': time_interval}})
# patch = list(result.values())[-1]
# #patch.save('patch/patches-delhi-data', overwrite_permission=OverwritePermission.OVERWRITE_FEATURES)

In [312]:
patch = []
patch += [EOPatch.load("patch/patches-delhi-data")]
patch += [EOPatch.load("patch/patches-hyderabad-data")]
patch += [EOPatch.load("patch/patches-chennai-data")]
patch += [EOPatch.load("patch/patches-bangalore-data")]
patch += [EOPatch.load("patch/patches-kolkata-data")]
patch += [EOPatch.load("patch/patches-mumbai-data")]

In [46]:
def plot_with_data(patch_data, component, dam_bbox, idx, plot_size):
    components_to_color = {"O3":["Blues",0,1],"SO2":"Reds", "CH4":"Greys", "NO2":"YlOrBr", "HCHO":"RdPu", "CO":"Purples", "AER_AI_354_388":"Greens"}
    ratio = np.abs(patch_data.bbox.max_x - patch_data.bbox.min_x) / np.abs(patch_data.bbox.max_y - patch_data.bbox.min_y)
    fig, ax = plt.subplots(figsize=(ratio * plot_size, plot_size))
    size = fig.get_size_inches()*fig.dpi
    img = Image.fromarray(np.squeeze(patch_data.data[component][idx]), 'F')
    img = img.resize((int(size[0]), int(size[1])))   
    patch_map = get_map_by_bbox(dam_bbox,int(size[0]), int(size[1]))
    patch_map = patch_map.resize((int(size[0]), int(size[1])))
    ax.set_title(component +" ~ "+ patch_data.timestamp[idx].strftime("%d/%m/%Y, %H:%M:%S"))
    ax.imshow(patch_map.convert('RGB'))
    im = ax.imshow(img,cmap=components_to_color[component], alpha=0.5)
    
    cax = fig.add_axes([0.90,0.15,0.02,0.70])
    cb = fig.colorbar(im, cax=cax, orientation='vertical')
    ax.axis('off')

In [223]:
components = ["O3","SO2", "NO2", "CO", "AER_AI_354_388"]

In [207]:
## Making all the Nan to Min values in the array
for i in patch.data:
    patch.data[i] = np.nan_to_num(patch.data[i],np.nanmin(patch.data[i]))

In [218]:
## mol/m2 to ppm conversion !!Not verified
def ppm(mpm2):
    return((mpm2) * (8.314*273.15) * 1000000 / 101300)

In [268]:
for i in range(0,5):
    print(patch[i].data["NO2"].max())-0.00051*	0.0192

0.0006093325
0.00010307087
0.0003305963
0.0002965223
0.00015897719


In [272]:
for i in range(0,5):
    print("{:f}".format(patch[i].data["NO2"].min()),":","{:f}".format(patch[i].data["NO2"].max()))

0.000000 : 0.000609
-0.000004 : 0.000103
-0.000001 : 0.000331
-0.000002 : 0.000297
-0.000012 : 0.000159


In [303]:
## To Verify if satellite values are in range
for i in range(len(patch)):
    for j in components:
        patch[i].data[j] = np.nan_to_num(patch[i].data[j],np.nanmin(patch[i].data[j]))
        #print(j,":",patch[i].data[j].min(),",",patch[i].data[j].max())
        print(j,":",patch[i].data[j].min()>=aq_values[j][0][0],",",patch[i].data[j].max()<aq_values[j][0][-1])

O3 : True , True
SO2 : True , True
NO2 : True , True
CO : True , True
AER_AI_354_388 : True , True
O3 : True , True
SO2 : True , True
NO2 : True , True
CO : True , True
AER_AI_354_388 : True , True
O3 : True , True
SO2 : True , True
NO2 : True , True
CO : True , True
AER_AI_354_388 : True , True
O3 : True , True
SO2 : True , True
NO2 : True , True
CO : True , True
AER_AI_354_388 : True , True
O3 : True , True
SO2 : True , True
NO2 : True , True
CO : True , True
AER_AI_354_388 : True , True
O3 : True , True
SO2 : True , True
NO2 : True , True
CO : True , True
AER_AI_354_388 : True , True


In [ ]:
for i in components:
    print(i,":",patch3.data[i].min()>aq_values[i][0][0],",",patch3.data[i].max()<aq_values[i][0][-1])

In [330]:
def find_quality(satellite_value,component):
    aq_values = {"NO2":[[-0.0005, 0.0045, 0.015, 0.02],0.01,"mol/m2"], 
    "SO2": [[-0.5,0.0593,0.625,2],0.35,"DU"],
    "O3": [[0,0.05,0.10,0.175], 0.075, "mol/m2"],
    "CO": [[0,0.04,0.06,0.1], 0.05, "mol/m2" ],
    "AER_AI_354_388": [[-5,-1,1,5],0, "Index Value"]}
    if(satellite_value>=aq_values[component][0][0] and satellite_value<aq_values[component][0][1]):
        return 0
    elif(satellite_value>aq_values[component][0][1] and satellite_value<aq_values[component][0][2]):
        return 1
    elif(satellite_value>aq_values[component][0][2] and satellite_value<aq_values[component][0][3]):
        return 2
    else:
        return -1

In [331]:
def aqi(patch):
    calculated_values = {"O3":[],"SO2":[], "NO2":[], "CO":[], "AER_AI_354_388":[]}
    for i in calculated_values:
        patch.data[i] = np.nan_to_num(patch.data[i],np.nanmin(patch.data[i]))
        quality_value_holder = []
        for j in range(len(patch.timestamp)):
            quality_value_holder += [find_quality(patch.data[i][j].max(),i)]
        calculated_values[i] = quality_value_holder
    return calculated_values

In [333]:
delhi_aqi = aqi(patch[0])

In [334]:
delhi_aqi

{'O3': [2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  0,
  2,
  2],
 'SO2': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'NO2': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'CO': [1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  2,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  0,
  0,
  0,
  0,
  2,
  2,
  1,
  1,
  2,
  2,
  0,
 